In [28]:
import boto3
import pandas as pd; pd.set_option('display.max_column', 100)
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, Lasso, LassoCV
from sklearn.ensemble import RandomForestClassifier, XGBoost


s3= boto3.resource('s3')
bucket_name= 'morgangant-bata-445-bucket'
bucket= s3.Bucket(bucket_name)

file_key= 'train.csv'

bucket_object= bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

#reading the datefile
project = pd.read_csv(file_content_stream)
project.head()

ImportError: cannot import name 'XGBoost' from 'sklearn.ensemble' (/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/ensemble/__init__.py)

In [3]:
#Dropping na values
project = project.dropna()

In [7]:
importances= list()
for i in range (1,100):
    ## Defining the input and taregt variables
    x= project[['trustLevel', 'totalScanTimeInSeconds', 'grandTotal', 'lineItemsVoids', 'scansWithoutRegistration', 'quanitityModification', 'scannedLineItemsPerSecond', 'valuePerSecond', 'lineItemVoidsPerPosition']]
    y= project['fraud']

    #Splitting data into train and test
    x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.2)

    #Building Model
    RF_md= RandomForestClassifier(n_estimators= 500).fit(x_train, y_train)

    #Extracting the feature importances
    importances.append(RF_md.feature_importances_)

In [8]:
a = pd.DataFrame(importances)
a.columns= ['trustLevel', 'totalScanTimeInSeconds', 'grandTotal', 'lineItemsVoids', 'scansWithoutRegistration', 'quanitityModification', 'scannedLineItemsPerSecond', 'valuePerSecond', 'lineItemVoidsPerPosition']
a.apply(np.mean, axis = 0)

trustLevel                   0.267337
totalScanTimeInSeconds       0.146603
grandTotal                   0.069167
lineItemsVoids               0.070835
scansWithoutRegistration     0.054783
quanitityModification        0.024785
scannedLineItemsPerSecond    0.166051
valuePerSecond               0.091405
lineItemVoidsPerPosition     0.109034
dtype: float64

In [8]:
md1_results= list()
md2_results= list()
md3_results= list()

#Define the input and target variable
x= project[['trustLevel', 'scannedLineItemsPerSecond', 'totalScanTimeInSeconds', 'lineItemVoidsPerPosition', 'valuePerSecond']]
y= project['fraud']
    

for i in range (0,1000):

    #splitting the data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, stratify=y)

    #Creating RF model with depth 3
    RF_md= RandomForestClassifier(max_depth= 3, n_estimators= 500).fit(x_train, y_train)
    #Predictingon the model
    RF_pred= RF_md.predict_proba(x_test)[:,1]
    RF_pred
    #Changling likleyhoods to labels
    RF_label= np.where(RF_pred < .1, 0, 1)
    #Computing recall
    Model1_recall= recall_score(y_test, RF_label)
    md1_results.append(Model1_recall)


    #Creating RF model with depth 5
    RF_md2= RandomForestClassifier(max_depth= 5, n_estimators= 500).fit(x_train, y_train)
    #Predictingon the model
    RF_pred2= RF_md2.predict_proba(x_test)[:,1]
    RF_pred2
    #Changling likleyhoods to labels
    RF_label2= np.where(RF_pred2 < .1, 0, 1)
    #Computing recall
    Model2_recall= recall_score(y_test, RF_label2)
    md2_results.append(Model2_recall)


    #Creating RF model with depth 5
    RF_md3= RandomForestClassifier(max_depth= 7, n_estimators= 500).fit(x_train, y_train)
    #Predictingon the model
    RF_pred3= RF_md3.predict_proba(x_test)[:,1]
    RF_pred3
    #Changling likleyhoods to labels
    RF_label3= np.where(RF_pred3 < .1, 0, 1)
    #Computing recall
    Model3_recall= recall_score(y_test, RF_label3)
    md3_results.append(Model3_recall)

KeyboardInterrupt: 

In [4]:
#Computing averages
print('Average Recall for model 1:', np.mean(md1_results))
print('Average Recall for model 2:',np.mean(md2_results))
print('Average Recall for model 3:',np.mean(md3_results))

Average Recall for model 1: 0.8852380952380952
Average Recall for model 2: 0.9371428571428569
Average Recall for model 3: 0.9414285714285714


In [9]:
#Trying different number of trees
md1_results= list()
md2_results= list()
md3_results= list()

#Define the input and target variable
x= project[['trustLevel', 'scannedLineItemsPerSecond', 'totalScanTimeInSeconds', 'lineItemVoidsPerPosition', 'valuePerSecond']]
y= project['fraud']
    
#depth of 7

for i in range (0,1000):

    #splitting the data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, stratify=y)

    #Creating RF model with 100 estimators
    RF_md= RandomForestClassifier(max_depth= 7, n_estimators= 100).fit(x_train, y_train)
    #Predictingon the model
    RF_pred= RF_md.predict_proba(x_test)[:,1]
    RF_pred
    #Changling likleyhoods to labels
    RF_label= np.where(RF_pred < .1, 0, 1)
    #Computing recall
    Model1_recall= recall_score(y_test, RF_label)
    md1_results.append(Model1_recall)


    #Creating RF model with 300 estimators
    RF_md2= RandomForestClassifier(max_depth= 7, n_estimators= 300).fit(x_train, y_train)
    #Predictingon the model
    RF_pred2= RF_md2.predict_proba(x_test)[:,1]
    RF_pred2
    #Changling likleyhoods to labels
    RF_label2= np.where(RF_pred2 < .1, 0, 1)
    #Computing recall
    Model2_recall= recall_score(y_test, RF_label2)
    md2_results.append(Model2_recall)


    #Creating RF model with 500 estimators
    RF_md3= RandomForestClassifier(max_depth= 7, n_estimators= 500).fit(x_train, y_train)
    #Predictingon the model
    RF_pred3= RF_md3.predict_proba(x_test)[:,1]
    RF_pred3
    #Changling likleyhoods to labels
    RF_label3= np.where(RF_pred3 < .1, 0, 1)
    #Computing recall
    Model3_recall= recall_score(y_test, RF_label3)
    md3_results.append(Model3_recall)

In [10]:
#Computing averages
print('Average Recall for model 1:', np.mean(md1_results))
print('Average Recall for model 2:',np.mean(md2_results))
print('Average Recall for model 3:',np.mean(md3_results))

Average Recall for model 1: 0.9393333333333335
Average Recall for model 2: 0.9387619047619048
Average Recall for model 3: 0.9390952380952382


In [14]:
#Trying different number of trees
md1_results= list()
md2_results= list()
md3_results= list()

#Define the input and target variable
x= project[['trustLevel', 'scannedLineItemsPerSecond', 'totalScanTimeInSeconds', 'lineItemVoidsPerPosition', 'valuePerSecond']]
y= project['fraud']
    
#depth of 3
    
for i in range (0,1000):

    #splitting the data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, stratify=y)

    #Creating RF model with 100 estimators
    RF_md= RandomForestClassifier(max_depth= 3, n_estimators= 100).fit(x_train, y_train)
    #Predictingon the model
    RF_pred= RF_md.predict_proba(x_test)[:,1]
    RF_pred
    #Changling likleyhoods to labels
    RF_label= np.where(RF_pred < .1, 0, 1)
    #Computing recall
    Model1_recall= recall_score(y_test, RF_label)
    md1_results.append(Model1_recall)


    #Creating RF model with with 300 estimators
    RF_md2= RandomForestClassifier(max_depth= 3, n_estimators= 300).fit(x_train, y_train)
    #Predictingon the model
    RF_pred2= RF_md2.predict_proba(x_test)[:,1]
    RF_pred2
    #Changling likleyhoods to labels
    RF_label2= np.where(RF_pred2 < .1, 0, 1)
    #Computing recall
    Model2_recall= recall_score(y_test, RF_label2)
    md2_results.append(Model2_recall)


    #Creating RF model with 500 estimators
    RF_md3= RandomForestClassifier(max_depth= 3, n_estimators= 500).fit(x_train, y_train)
    #Predictingon the model
    RF_pred3= RF_md3.predict_proba(x_test)[:,1]
    RF_pred3
    #Changling likleyhoods to labels
    RF_label3= np.where(RF_pred3 < .1, 0, 1)
    #Computing recall
    Model3_recall= recall_score(y_test, RF_label3)
    md3_results.append(Model3_recall)

In [15]:
#Computing averages
print('Average Recall for model 1:', np.mean(md1_results))
print('Average Recall for model 2:',np.mean(md2_results))
print('Average Recall for model 3:',np.mean(md3_results))

Average Recall for model 1: 0.8792380952380954
Average Recall for model 2: 0.8787619047619049
Average Recall for model 3: 0.8791428571428573


In [16]:
#Trying different number of trees
md1_results= list()
md2_results= list()
md3_results= list()

#Define the input and target variable
x= project[['trustLevel', 'scannedLineItemsPerSecond', 'totalScanTimeInSeconds', 'lineItemVoidsPerPosition', 'valuePerSecond']]
y= project['fraud']
    
#depth of 5
    
for i in range (0,1000):

    #splitting the data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, stratify=y)

    #Creating RF model with 100 estimators
    RF_md= RandomForestClassifier(max_depth= 5, n_estimators= 100).fit(x_train, y_train)
    #Predictingon the model
    RF_pred= RF_md.predict_proba(x_test)[:,1]
    RF_pred
    #Changling likleyhoods to labels
    RF_label= np.where(RF_pred < .1, 0, 1)
    #Computing recall
    Model1_recall= recall_score(y_test, RF_label)
    md1_results.append(Model1_recall)


    #Creating RF model with with 300 estimators
    RF_md2= RandomForestClassifier(max_depth= 5, n_estimators= 300).fit(x_train, y_train)
    #Predictingon the model
    RF_pred2= RF_md2.predict_proba(x_test)[:,1]
    RF_pred2
    #Changling likleyhoods to labels
    RF_label2= np.where(RF_pred2 < .1, 0, 1)
    #Computing recall
    Model2_recall= recall_score(y_test, RF_label2)
    md2_results.append(Model2_recall)


    #Creating RF model with 500 estimators
    RF_md3= RandomForestClassifier(max_depth=5, n_estimators= 500).fit(x_train, y_train)
    #Predictingon the model
    RF_pred3= RF_md3.predict_proba(x_test)[:,1]
    RF_pred3
    #Changling likleyhoods to labels
    RF_label3= np.where(RF_pred3 < .1, 0, 1)
    #Computing recall
    Model3_recall= recall_score(y_test, RF_label3)
    md3_results.append(Model3_recall)

In [17]:
#Computing averages
print('Average Recall for model 1:', np.mean(md1_results))
print('Average Recall for model 2:',np.mean(md2_results))
print('Average Recall for model 3:',np.mean(md3_results))

Average Recall for model 1: 0.9297619047619048
Average Recall for model 2: 0.9307619047619049
Average Recall for model 3: 0.9306666666666668


In [25]:
import xgboost as xgb 

results= list()

for i in range(0,100):
    #splitting the data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, stratify=y)
    #creating model
    xgb_md= xgb.XGBClassifier(max_depth= 3, n_estimators= 500, learning_rate= .1, ).fit(x_train, y_train)
    #predicitng on the model
    xgb_pred= xgb_md.predict_proba(x_test)[:,1]
    #Changling likleyhoods to labels
    xgb_label= np.where(xgb_pred < .1, 0, 1)
    #Computing recall and appending score
    xgb_recall= recall_score(y_test, xgb_label)
    results.append(xgb_recall)

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
#Computing averages
print('Average Recall for xgb model:', np.mean(results))